In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR


import geopandas as gpd
import numpy as np
import pandas as pd
from pykrige.ok import OrdinaryKriging
from pykrige.uk import UniversalKriging
from shapely.geometry import Point, Polygon
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import descartes
import math
import matplotlib.colors as mcolors
import pyproj
import folium

In [33]:
wdir = "../Dades/AMS_Observacions/"
crs_latlon = 'EPSG:4326'  # WGS84
crs_utm = "EPSG:32631"

In [26]:
# Load the dataset for contaminants

df_no2 = pd.read_csv('../Dades/AMS_Observacions/gene_sconcno2_2023_xvpca_emep_port.csv') 
df_estacions = pd.read_csv('../Dades/AMS_Observacions/XVPCA_info_sconcno2_2023.csv')

print(df_estacions.columns)

# Convert 'Date' column to datetime format
df_no2['Date'] = pd.to_datetime(df_no2['Date'])
print(df_no2.columns)
# Pivot the dataframe to have each station ID as a column and the Date as the index
# columnas_es = [col for col in df_no2.columns if col.startswith('ES')]
columnas_es =df_no2.drop(['Date'], axis=1)


# Usar melt para pivotar el DataFrame
df_melted = pd.melt(
    df_no2, 
    value_vars=columnas_es,      # Columnas de estaciones a convertir
    var_name='Station',          # Columna con los nombres de estaciones
    value_name='Value'           # Columna con los valores de cada estación
)

# Agregar las demás columnas como atributos (sin repetirlas)
df_melted['Date'] = df_no2['Date'].iloc[0]
# df_melted['ZAL'] = df_no2['ZAL'].iloc[0]

# Mostrar el resultado
print(df_melted)

nom_estacions = ['ZAL',
# Combina les taules utilitzant 'UniqueID' com a clau de combinació
# combined_data = pd.merge(categorical_data, acoustic_features, on='UniqueID', how='inner'),

Index(['code', 'lat', 'lon', 'type'], dtype='object')
Index(['Date', 'ZAL', 'ES1438A', 'ES1480A', 'ES1396A', 'ES1992A', 'ES0691A',
       'ES0692A', 'ES1148A', 'ES1679A', 'ES1856A', 'ES2090A', 'ES1453A',
       'ES1929A', 'ES1903A', 'ES1910A', 'ES2011A', 'ES2012A', 'ES1684A',
       'ES1231A', 'ES1891A', 'ES0584A', 'ES1262A', 'ES1018A', 'ES1551A',
       'ES1814A', 'ES1817A', 'ES1126A', 'ES0971A', 'ES1339A', 'ES1815A',
       'ES1773A', 'ES1899A', 'ES1122A', 'ES1123A', 'ES2109A', 'ES1124A',
       'ES1208A', 'ES1666A', 'ES1312A', 'ES1812A', 'ES1125A', 'ES1135A',
       'ES1642A', 'ES1397A', 'ES1923A', 'ES1816A', 'ES1999A', 'ES1347A',
       'ES1778A', 'ES1275A', 'ES1311A', 'ES1851A', 'ES1348A', 'ES1982A',
       'ES1225A', 'ES2034A', 'ES1215A', 'ES2017A', 'ES1930A', 'ES1948A',
       'ES1855A', 'ES1854A', 'ES0010R', 'ES0014R'],
      dtype='object')
        Station  Value                Date  ZAL
0       ES1438A    NaN 2022-12-31 23:00:00  NaN
1       ES1438A    NaN 2022-12-31 23:00:00

In [18]:
df_comb = pd.merged(df_melted, df_estacions, on= 'code', how = 'inner')

df_comb.head()

    

In [36]:
df_no2 = pd.read_csv('../Dades/AMS_Observacions/gene_sconcno2_2023_xvpca_emep_port.csv') 
df_estacions = pd.read_csv('../Dades/AMS_Observacions/XVPCA_info_sconcno2_2023.csv')


# Convert 'Date' column to datetime format
df_no2['Date'] = pd.to_datetime(df_no2['Date'])

# Pivot the dataframe to have each station ID as a column and the Date as the index
# columnas_es = [col for col in df_no2.columns if col.startswith('ES')]
columnas_es =df_no2.drop(['Date'], axis=1)


# Transpose the dataframe to have each station ID as a row and the Date as the column
df_no2_transposed = df_no2.set_index('Date').transpose().reset_index()

# Melt the dataframe to have a 'Value' column for each hour
df_no2_melted = pd.melt(df_no2_transposed, id_vars=['index'], var_name='Date', value_name='no2')

# Rename the 'index' column to 'Station'
df_no2_melted.rename(columns={'index': 'Station'}, inplace=True)

# Drop rows with NaN values
df_no2_melted.dropna(subset=['no2'], inplace=True)

print(df_no2_melted)

# Join the melted dataframe with the station information dataframe
df_obs = pd.merge(df_no2_melted, df_estacions, left_on='Station', right_on='code', how='inner')

# Drop the 'code' column as it is redundant
df_obs.drop(columns=['code','type'], inplace=True)
df_obs['geometry'] = gpd.points_from_xy(df_obs['lon'], df_obs['lat'], crs=crs_utm)
print(df_obs)

        Station                 Date   no2
2       ES1480A  2022-12-31 23:00:00  62.0
3       ES1396A  2022-12-31 23:00:00  60.0
4       ES1992A  2022-12-31 23:00:00  45.0
5       ES0691A  2022-12-31 23:00:00  43.0
6       ES0692A  2022-12-31 23:00:00  45.0
...         ...                  ...   ...
560633  ES2017A  2023-12-31 22:00:00   7.0
560634  ES1930A  2023-12-31 22:00:00   1.0
560635  ES1948A  2023-12-31 22:00:00   3.0
560636  ES1855A  2023-12-31 22:00:00   2.0
560637  ES1854A  2023-12-31 22:00:00   1.0

[529219 rows x 3 columns]


KeyError: 'lon'